## Trabajo Práctico Nº 4
### Referido a  base de datos vectoriales. chromadb.

In [ ]:
#!pip install transformers==4.35.0 sentence-transformers==2.2.2 chromadb==0.4.6 accelerate tqdm 

In [ ]:
#!pip install transformers==4.35.0 sentence-transformers==2.2.2 chromadb==0.4.6 accelerate tqdm 

# Resolución

## Comentarios previos
Dado la gran cantidad de errores que tuve con mi entorno de transformers en mac, opté por usar la librería de open ia para los embedding. El concepto es el mismo, por lo que pude avanzar logrando el resultado esperado.

## Código base

In [1]:
# 1️⃣ Importar librerías
import openai
import chromadb
from chromadb.config import Settings
import os
import uuid

In [28]:
# Configurar la clave de API de OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")  # Usa una variable de entorno para mayor seguridad

# Función para obtener embeddings usando OpenAI
def get_embeddings(text):
    try:
        response = openai.embeddings.create(
            input=text,
            model="text-embedding-ada-002"  # Modelo de OpenAI para embeddings
        )
        return response.data[0].embedding
    except openai.error.OpenAIError as e:
        print(f"❌ Error al obtener embeddings: {e}")
        return None

# Inicializar ChromaDB con persistencia
client = chromadb.PersistentClient(path="./chroma_store")
collection = client.get_or_create_collection(name="documentos_ia")

def cargar_documentos(base_path="/Users/fabricio.denardi/Documents/CEIA/BDIA/repos/MIA_01c_BDIA/TP4"):
    docs_path = os.path.join(base_path, "docs")
    documentos = []
    metadatos = []
    ids = []

    for filename in os.listdir(docs_path):
        if filename.endswith(".txt"):
            with open(os.path.join(docs_path, filename), 'r', encoding="utf-8") as f:
                text = f.read().strip()
                chunks = [text[i:i+500] for i in range(0, len(text), 500)]
                
                for i, chunk in enumerate(chunks):
                    documentos.append(chunk)
                    metadatos.append({"source": filename})
                    ids.append(str(uuid.uuid4()))  # ID único
    
    return documentos, metadatos, ids

def crear_base_datos():
    docs, metas, ids = cargar_documentos()
    embeddings = [get_embeddings(d) for d in docs if get_embeddings(d) is not None]
    collection.add(documents=docs, embeddings=embeddings, metadatas=metas, ids=ids)
    print("✅ Base de datos creada con éxito.")

# --- CRUD FUNCIONES ---

def create_example(new_doc):
    """Añadir un documento nuevo"""
    embedding = get_embeddings(new_doc)
    if embedding:
        doc_id = str(uuid.uuid4())
        metadata = {"source": "nuevo_documento.txt"}
        
        collection.add(
            documents=[new_doc],
            embeddings=[embedding],
            metadatas=[metadata],
            ids=[doc_id]
        )
        print(f"✅ Documento añadido con ID: {doc_id}")
    else:
        print("❌ No se pudo añadir el documento debido a un error en los embeddings.")

def read_example(query, n_results = 1):
    """Consultar documentos similares"""
    query_emb = get_embeddings(query)
    if query_emb:
        results = collection.query(
            query_embeddings=[query_emb],
            n_results=n_results
        )
        
        if results["documents"]:
            for i in range(n_results):
                print(f"📄 Mejor coincidencia nro {i}: {results['documents'][i-1][0]}")
                print(f"📁 Fuente: {results['metadatas'][i-1][0]['source']}")
                print(f"🆔 ID: {results['ids'][i-1][0]}")
        else:
            print("⚠️ No se encontraron resultados.")
    else:
        print("❌ No se pudo realizar la consulta debido a un error en los embeddings.")

def get_documents_by_id(doc_id):
    """Obtener un documento por ID"""
    results = collection.get(ids=[doc_id], include=["documents", "metadatas", "embeddings"])
    
    if results["documents"]:
        print(f"📄 Documento: {results['documents'][0]}")
        print(f"📁 Fuente: {results['metadatas'][0]['source']}")
    else:
        print("⚠️ Documento no encontrado.")

def update_example(old_id, new_text):
    """Actualizar un documento (eliminar y reinsertar)"""
    collection.delete(ids=[old_id])
    
    new_id = str(uuid.uuid4())
    embedding = get_embeddings(new_text)
    if embedding:
        metadata = {"source": "documento_actualizado.txt"}
        
        collection.add(
            documents=[new_text],
            embeddings=[embedding],
            metadatas=[metadata],
            ids=[new_id]
        )
        print(f"♻️ Documento actualizado. Nuevo ID: {new_id}")
    else:
        print("❌ No se pudo actualizar el documento debido a un error en los embeddings.")

def delete_example(doc_id):
    """Eliminar por ID"""
    collection.delete(ids=[doc_id])
    print(f"🗑️ Documento con ID '{doc_id}' eliminado.")

print('listo')

listo


In [10]:
crear_base_datos()

✅ Base de datos creada con éxito.


In [18]:
get_embeddings("El sol se alzaba sobre las colinas de Buenos Aires, pintando el cielo con tonos naranja y rosa.")

[-0.003770906012505293,
 -0.006599085405468941,
 -0.023411041125655174,
 -0.00565308565273881,
 0.015083624050021172,
 -0.00786260049790144,
 -0.001582667580805719,
 -0.021014945581555367,
 -0.0038461931981146336,
 -0.03006250038743019,
 0.01929970644414425,
 0.029931565746665,
 -0.01020632404834032,
 -0.02008531242609024,
 -0.010815167799592018,
 0.02229810133576393,
 0.020491208881139755,
 0.015188371762633324,
 0.020936384797096252,
 -0.010396178811788559,
 -0.01953538879752159,
 0.009283237159252167,
 0.010893728584051132,
 -0.015869230031967163,
 -0.028831718489527702,
 0.023803845047950745,
 0.00034063358907587826,
 -0.0022357020061463118,
 0.0002036641089944169,
 0.016811955720186234,
 0.027417629957199097,
 -0.006608905736356974,
 -0.0022242453414946795,
 -0.007024621590971947,
 -0.023306293413043022,
 0.013577880337834358,
 -0.003291359404101968,
 0.0004689082270488143,
 0.020163873210549355,
 -0.00867439340800047,
 0.01640605926513672,
 -0.01846172660589218,
 0.00084534415509

In [11]:
results = collection.get(include=["embeddings", "documents", "metadatas"])

In [12]:
for id, content, metadata, embedding in zip(results["ids"], results["documents"], results["metadatas"], results["embeddings"]):
    print(f"ID: {id}, Contenido: {content[:100]} ", f"Metadata: {metadata}", f"Embedding: {embedding[:5]}...")

ID: 13e72961-a0db-4709-8644-af65868394ea, Contenido: El sol se alzaba sobre las colinas de Buenos Aires, pintando el cielo con tonos naranja y rosa. El a  Metadata: {'source': 'doc1.txt'} Embedding: [0.002886376576498151, 0.0004124291008338332, -0.019652780145406723, -0.027064379304647446, 0.002653928007930517]...
ID: 1304f150-8c1c-4669-b0dc-30c62db3aa86, Contenido: ciosa continuaba, con el tráfico constante y el murmullo de las conversaciones. Los turistas se mara  Metadata: {'source': 'doc1.txt'} Embedding: [-0.011876767501235008, -0.010162802413105965, 0.017784904688596725, -0.010875274427235126, 0.008072437718510628]...
ID: efde7866-808d-4c13-a277-8db4f8bc256d, Contenido:  los restaurantes servían deliciosas parrilladas.

Hacia el norte, el elegante barrio de Recoleta in  Metadata: {'source': 'doc1.txt'} Embedding: [-0.004808980040252209, 6.465915066655725e-05, 0.01111509557813406, -0.008399512618780136, 0.013895489275455475]...
ID: 619ccb53-f5bd-4435-815e-3e21249835e8, Contenido: 

In [21]:
while True:
    opcion = input("Elige una opción o escribe ayuda: ")
    
    if opcion=="ayuda":
        print("\n--- Menú CRUD ---")
        print("1. Crear documento")
        print("2. Consultar por tema")
        print("3. Actualizar documento (por ID)")
        print("4. Eliminar documento (por ID)")
        print("5. Buscar documento (por ID)")
        print("6. Salir")

    if opcion == "1":
        nuevo_doc = input("Ingresa el texto a añadir: ")
        create_example(nuevo_doc)
        
    elif opcion == "2":
        consulta = input("Consulta: ")
        read_example(consulta)
        
    elif opcion == "3":
        doc_id = input("ID del documento a actualizar: ")
        nuevo_texto = input("Nuevo contenido: ")
        update_example(doc_id, nuevo_texto)
        
    elif opcion == "4":
        doc_id = input("ID del documento a eliminar: ")
        delete_example(doc_id)  

    elif opcion == "5":
        doc_id = input("ID del documento a buscar: ")
        get_documents_by_id(doc_id)
        
    elif opcion == "6":
        print("👋 Saliendo del programa.")
        break

👋 Saliendo del programa.


In [24]:
# Esta sección es para la integración con LM Studio y no es parte del CRUD. Es solo un ejemplo de cómo se podría hacer una consulta a un modelo de lenguaje como Llama 3 o similar.
# 2️⃣ Integración con LM Studio (Llama 3 o similar)
import requests
import json

LM_STUDIO_URL = "http://127.0.0.1:1234/v1/chat/completions"  

def query_llama(prompt):
    """Envia un prompt a LM Studio y recibe la respuesta."""
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 256, 
        # Es posible agregar otros parámetros de generación aca (temperature, top_p, etc.)
    }
    try:
        response = requests.post(LM_STUDIO_URL, headers=headers, data=json.dumps(data))
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        print(f"Error al conectar con LM Studio: {e}")
        return None
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Error al procesar la respuesta de LM Studio: {e}")
        return None

def responder_pregunta(pregunta):
    """Busca documentos relevantes y luego consulta a Llama."""
    query_emb = get_embeddings(pregunta)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=3  # Obtener algunos documentos relevantes como contexto
    )

    if results["documents"]:
        contexto = "\n\n".join(results["documents"][0])
        prompt = f"Basándote en la siguiente información:\n\n{contexto}\n\nResponde a la siguiente pregunta: {pregunta}" # Juntar el contexto obtenido y la pregunta
        respuesta = query_llama(prompt)
        return respuesta
    else:
        return "No se encontraron documentos relevantes para responder a tu pregunta."

# --- Ejemplo de uso ---
if __name__ == "__main__":
    crear_base_datos() 

    while True:
        pregunta_usuario = input("Pregúntame algo sobre los documentos (o escribe 'salir'): ")
        if pregunta_usuario.lower() == 'salir':
            break

        print(f"🤖 Pregunta: {pregunta_usuario}")
        respuesta_ia = responder_pregunta(pregunta_usuario)
        if respuesta_ia:
            print(f"🤖 Respuesta de la IA: {respuesta_ia}")

✅ Base de datos creada con éxito.
🤖 Pregunta: based on the context, give me the step by step to prepare a bechamel 
🤖 Respuesta de la IA: <think>
Okay, I need to figure out how to make Béchamel sauce based on the information provided in the issue description. Let's see what's given.

First, looking at the ingredients listed:
- 50g butter
- 50g flour
- 500ml milk
- Seasoning (white pepper preferred)
- 100g cheddar cheese
- Nutmeg

The method steps provided are partial. The user started by melting the butter, adding flour and stirring until a dough is formed. Then in step 3, it says "gradually add the," but that's cut off.

I think step 3 might be adding something else. Since there's cheddar cheese mentioned in ingredients, perhaps they meant to add the cheese gradually as well. Alternatively, maybe they forgot part of the method after step 2.

Let me recall how Béchamel is made. Typically, you start by melting butter and flour to make a roux, then slowly add milk while stirring until it

## Resolución de las preguntas

### Ejercicio 1: Crear y Consultar 
 
Crea un documento nuevo con el texto: "La gravedad es una fuerza fundamental en el 
universo". 
Realiza una consulta para encontrar documentos similares a la frase: "Fuerzas naturales 
que gobiernan el cosmos". 
 
¿Qué tipo de operación CRUD usaste primero? 
¿Cuál es el ID del documento creado? (Usa collection.peek()). 
Explica por qué el documento aparece como coincidencia en la consulta.

In [13]:
# Crear un documento nuevo
nuevo_texto = "La gravedad es una fuerza fundamental en el universo"
create_example(nuevo_texto)

# Consultar documentos similares
consulta = "Fuerzas naturales que gobiernan el cosmos"
read_example(consulta)

# Verificar el ID del documento creado
peek_result = collection.peek()
print(f'ID del documento creado: {peek_result["ids"][-1]}')

# Explicación
print("El documento aparece como coincidencia porque el modelo de embeddings identifica similitudes semánticas entre el texto del documento y la consulta.")

✅ Documento añadido con ID: da07e69f-3b9c-456c-a479-1f70001bc82f
📄 Mejor coincidencia: La gravedad es una fuerza fundamental en el universo
📁 Fuente: nuevo_documento.txt
🆔 ID: da07e69f-3b9c-456c-a479-1f70001bc82f
ID del documento creado: 13e72961-a0db-4709-8644-af65868394ea
El documento aparece como coincidencia porque el modelo de embeddings identifica similitudes semánticas entre el texto del documento y la consulta.


### Ejercicio 2: Actualizar y Eliminar 
 
Crea un documento con el texto: "Redes neuronales imitan procesos cerebrales". 
Encuentra su ID, luego actualízalo por: "Deep Learning es un subcampo de IA con redes 
neuronales profundas". 
Elimina el documento original usando el mismo ID. 
 
¿Qué ocurre si intentas actualizar un documento eliminado? 
¿Cómo se garantiza la integridad de los datos en las operaciones?

In [14]:
# Crear un documento nuevo
nuevo_doc = "Redes neuronales imitan procesos cerebrales"
create_example(nuevo_doc)

# Encontrar su ID
peek_result = collection.peek()
doc_id = peek_result['ids'][-1]  # Asumimos que el último documento añadido es el nuevo
print(f"ID del documento creado: {doc_id}")

# Actualizar el documento
nuevo_texto = "Deep Learning es un subcampo de IA con redes neuronales profundas"
update_example(doc_id, nuevo_texto)

# Eliminar el documento original usando el mismo ID
delete_example(doc_id)

# Intentar actualizar un documento eliminado
try:
    update_example(doc_id, "Intento de actualización tras eliminación")
except Exception as e:
    print(f"Error al intentar actualizar un documento eliminado: {e}")

# Explicación sobre la integridad de los datos
print("La integridad de los datos se garantiza mediante el uso de IDs únicos para cada documento. "
      "Las operaciones como actualización y eliminación verifican la existencia del documento antes de proceder, "
      "asegurando que no se realicen cambios en documentos inexistentes.")

Delete of nonexisting embedding ID: 00236458-4f48-4598-88d2-33bce6f0f37b


✅ Documento añadido con ID: 369642d9-ebb8-406a-8ef7-95e39ed67fc9
ID del documento creado: 00236458-4f48-4598-88d2-33bce6f0f37b


Delete of nonexisting embedding ID: 00236458-4f48-4598-88d2-33bce6f0f37b
Delete of nonexisting embedding ID: 00236458-4f48-4598-88d2-33bce6f0f37b
Delete of nonexisting embedding ID: 00236458-4f48-4598-88d2-33bce6f0f37b
Delete of nonexisting embedding ID: 00236458-4f48-4598-88d2-33bce6f0f37b


♻️ Documento actualizado. Nuevo ID: 1e6645fc-cee4-429d-82bb-c13ebbe4e85c
🗑️ Documento con ID '00236458-4f48-4598-88d2-33bce6f0f37b' eliminado.
♻️ Documento actualizado. Nuevo ID: ff301c43-46a3-4e2b-be24-6f23b59ec2eb
La integridad de los datos se garantiza mediante el uso de IDs únicos para cada documento. Las operaciones como actualización y eliminación verifican la existencia del documento antes de proceder, asegurando que no se realicen cambios en documentos inexistentes.


### Ejercicio 3: Errores y Validación 
Intenta eliminar un documento con el ID "no_existe". 
Crea un nuevo documento sin usar get_embeddings() (ejemplo: enviar un array vacío). 
 
¿Qué errores ocurren en cada caso? 
¿Cómo prevenir estos problemas?

In [15]:
# Intentar eliminar un documento con un ID inexistente
try:
    collection.delete(ids=["no_existe"])
    print("🗑️ Documento eliminado.")
except Exception as e:
    print(f"❌ Error al intentar eliminar un documento inexistente: {e}")

# Intentar crear un documento sin usar get_embeddings()
try:
    collection.add(
        documents=["Este es un documento sin embeddings."],
        embeddings=[],  # Array vacío
        metadatas=[{"source": "sin_embeddings.txt"}],
        ids=[str(uuid.uuid4())]
    )
    print("✅ Documento creado sin embeddings.")
except Exception as e:
    print(f"❌ Error al intentar crear un documento sin embeddings: {e}")

# Prevención de problemas
print("\n--- Prevención de problemas ---")
print("1. Verificar la existencia del ID antes de intentar eliminarlo.")
print("2. Asegurarse de generar embeddings válidos antes de añadir documentos.")
print("3. Implementar validaciones en el código para manejar errores de forma adecuada.")

Delete of nonexisting embedding ID: no_existe
Delete of nonexisting embedding ID: no_existe


🗑️ Documento eliminado.
❌ Error al intentar crear un documento sin embeddings: list index out of range

--- Prevención de problemas ---
1. Verificar la existencia del ID antes de intentar eliminarlo.
2. Asegurarse de generar embeddings válidos antes de añadir documentos.
3. Implementar validaciones en el código para manejar errores de forma adecuada.


### Ejercicio 4: Consultas por Tema 
 Crea tres documentos: 
"Python es ideal para ciencia de datos" 
"JavaScript maneja interacciones web dinámicas" 
"Java se usa en aplicaciones empresariales" 
Realiza una consulta por la frase: "Lenguajes backend y frontend". 
 
¿Cuáles documentos aparecen como coincidencias? 
Analiza cómo las palabras clave influyen en los resultados.

In [16]:
# Crear los tres documentos
create_example("Python es ideal para ciencia de datos")
create_example("JavaScript maneja interacciones web dinámicas")
create_example("Java se usa en aplicaciones empresariales")

# Realizar la consulta
consulta = "Lenguajes backend y frontend"
read_example(consulta)

# Explicación
print("Los documentos que aparecen como coincidencias son aquellos cuyos embeddings tienen mayor similitud semántica con la consulta.")
print("Las palabras clave como 'backend', 'frontend', 'interacciones web' o 'aplicaciones empresariales' influyen en los resultados, ya que el modelo de embeddings evalúa la relación semántica entre los términos.")

✅ Documento añadido con ID: 3b2f2c23-f52d-4bbe-a4ac-e6428c2b3af9
✅ Documento añadido con ID: 06c20714-f604-4ef6-8acc-03bc0ef67e8a
✅ Documento añadido con ID: b7c64d38-d38d-4cc3-9218-7307d4021b9d
📄 Mejor coincidencia: JavaScript maneja interacciones web dinámicas
📁 Fuente: nuevo_documento.txt
🆔 ID: 06c20714-f604-4ef6-8acc-03bc0ef67e8a
Los documentos que aparecen como coincidencias son aquellos cuyos embeddings tienen mayor similitud semántica con la consulta.
Las palabras clave como 'backend', 'frontend', 'interacciones web' o 'aplicaciones empresariales' influyen en los resultados, ya que el modelo de embeddings evalúa la relación semántica entre los términos.


### Ejercicio 5: Eliminaciones Selectivas 
Crea dos documentos con tópicos opuestos (ejemplo: uno sobre "Ecología" y otro sobre "Minería de datos"). 
Elimina el documento sobre ecología usando su ID. 
Verifica que solo quede un documento. 
 
¿Cómo usar collection.count() para confirmar?

In [17]:
# Crear dos documentos con tópicos opuestos
create_example("La ecología estudia las relaciones entre los seres vivos y su entorno.")
create_example("La minería de datos permite extraer patrones útiles de grandes volúmenes de información.")

# Obtener los IDs de los documentos creados
peek_result = collection.peek()
doc_id_ecologia = peek_result['ids'][-2]  # Asumimos que el penúltimo documento es el de ecología
doc_id_mineria = peek_result['ids'][-1]  # Asumimos que el último documento es el de minería de datos

# Eliminar el documento sobre ecología usando su ID
delete_example(doc_id_ecologia)

# Verificar que solo quede un documento
remaining_count = collection.count()
print(f"📊 Número de documentos restantes en la colección: {remaining_count}")

✅ Documento añadido con ID: 1aceca30-7442-4920-bcf9-3ce041461fc8


Delete of nonexisting embedding ID: c7241ba9-5f07-4989-aa0a-05a44f7fc372


✅ Documento añadido con ID: fe11b74f-4270-4092-a9c9-344aaa759031
🗑️ Documento con ID 'c7241ba9-5f07-4989-aa0a-05a44f7fc372' eliminado.
📊 Número de documentos restantes en la colección: 1368


### Ejercicio 6: Actualización en Cadena 
 Crea un documento con el texto "IA y ética son temas actuales". 
Actualízalo dos veces: 
Primera actualización: "Ética en IA aborda dilemas morales" 
Segunda actualización: "El uso de IA debe estar regulado por principios éticos". 
 
¿Cómo sigue evolucionando el contenido del documento con cada update? 

In [18]:
# Crear un documento con el texto inicial
texto_inicial = "IA y ética son temas actuales"
create_example(texto_inicial)

# Obtener el ID del documento recién creado
peek_result = collection.peek()
doc_id_ia_etica = peek_result['ids'][-1]  # Asumimos que el último documento añadido es el nuevo
print(f"ID del documento creado: {doc_id_ia_etica}")

# Primera actualización
texto_actualizado_1 = "Ética en IA aborda dilemas morales"
update_example(doc_id_ia_etica, texto_actualizado_1)

# Segunda actualización
texto_actualizado_2 = "El uso de IA debe estar regulado por principios éticos"
update_example(doc_id_ia_etica, texto_actualizado_2)

# Explicación
print("El contenido del documento evoluciona con cada actualización, reflejando los cambios realizados. "
    "Cada vez que se actualiza, el documento original se elimina y se reemplaza con una nueva versión, "
    "manteniendo un nuevo ID único para garantizar la integridad de los datos.")

Delete of nonexisting embedding ID: 1e391d2a-b36f-4128-917d-b5d1c8b78041


✅ Documento añadido con ID: 9f36ad20-e6a2-429f-9b46-815d203096e0
ID del documento creado: 1e391d2a-b36f-4128-917d-b5d1c8b78041


Delete of nonexisting embedding ID: 1e391d2a-b36f-4128-917d-b5d1c8b78041
Delete of nonexisting embedding ID: 1e391d2a-b36f-4128-917d-b5d1c8b78041


♻️ Documento actualizado. Nuevo ID: fa33c884-b0f3-4bb2-8ea5-47f058166920
♻️ Documento actualizado. Nuevo ID: e55472a8-8a20-4250-bba8-33ab7e9ecd2b
El contenido del documento evoluciona con cada actualización, reflejando los cambios realizados. Cada vez que se actualiza, el documento original se elimina y se reemplaza con una nueva versión, manteniendo un nuevo ID único para garantizar la integridad de los datos.


### Ejercicio 7: Consultas Ambiguas 
 
Crea dos documentos similares pero no idénticos: 
"Blockchain garantiza transacciones seguras" 
"Criptomonedas usan tecnología blockchain". 
Realiza una consulta por "Seguridad en finanzas digitales". 
 
¿Ambos documentos aparecen? Explica la similitud semántica. 

In [29]:
# Crear dos documentos similares pero no idénticos
create_example("Blockchain garantiza transacciones seguras")
create_example("Criptomonedas usan tecnología blockchain")

# Realizar la consulta
consulta = "Seguridad en finanzas digitales"
read_example(consulta)
read_example(consulta,n_results=2)
# Explicación
print("Ambos documentos pueden aparecer como coincidencias si el modelo de embeddings identifica una similitud semántica entre los términos 'blockchain', 'transacciones seguras', 'criptomonedas' y 'finanzas digitales'.")
print("La similitud semántica se basa en cómo el modelo de embeddings representa las palabras y frases en un espacio vectorial, evaluando la proximidad entre los vectores generados.")

Delete of nonexisting embedding ID: 00236458-4f48-4598-88d2-33bce6f0f37b
Delete of nonexisting embedding ID: 00236458-4f48-4598-88d2-33bce6f0f37b
Delete of nonexisting embedding ID: 00236458-4f48-4598-88d2-33bce6f0f37b
Delete of nonexisting embedding ID: no_existe
Delete of nonexisting embedding ID: c7241ba9-5f07-4989-aa0a-05a44f7fc372
Delete of nonexisting embedding ID: 1e391d2a-b36f-4128-917d-b5d1c8b78041
Delete of nonexisting embedding ID: 1e391d2a-b36f-4128-917d-b5d1c8b78041


✅ Documento añadido con ID: c0b4483d-29df-487b-8125-1a98ef3c11c4
✅ Documento añadido con ID: b3ee7a5d-b184-48d3-a854-22d718e8c02f
📄 Mejor coincidencia nro 0: Blockchain garantiza transacciones seguras
📁 Fuente: nuevo_documento.txt
🆔 ID: bd378554-ece7-4701-8bb8-c17aa64ded1d
📄 Mejor coincidencia nro 0: Blockchain garantiza transacciones seguras
📁 Fuente: nuevo_documento.txt
🆔 ID: bd378554-ece7-4701-8bb8-c17aa64ded1d
📄 Mejor coincidencia nro 1: Blockchain garantiza transacciones seguras
📁 Fuente: nuevo_documento.txt
🆔 ID: bd378554-ece7-4701-8bb8-c17aa64ded1d
Ambos documentos pueden aparecer como coincidencias si el modelo de embeddings identifica una similitud semántica entre los términos 'blockchain', 'transacciones seguras', 'criptomonedas' y 'finanzas digitales'.
La similitud semántica se basa en cómo el modelo de embeddings representa las palabras y frases en un espacio vectorial, evaluando la proximidad entre los vectores generados.


### Ejercicio 8: Reiniciar Base de Datos 
 
Elimina todos los documentos usando delete() con el parámetro correcto. 
Vuelve a cargar la base desde cero con crear_base_datos(). 
 
¿Cuántos documentos hay ahora? (Usa collection.count()).

In [30]:
# Eliminar todos los documentos de la colección
collection.delete()

# Volver a cargar la base desde cero
crear_base_datos()

# Contar cuántos documentos hay ahora
document_count = collection.count()
print(f"📊 Número de documentos en la colección después de recargar: {document_count}")

Delete of nonexisting embedding ID: 13e72961-a0db-4709-8644-af65868394ea
Delete of nonexisting embedding ID: 1304f150-8c1c-4669-b0dc-30c62db3aa86
Delete of nonexisting embedding ID: efde7866-808d-4c13-a277-8db4f8bc256d
Delete of nonexisting embedding ID: 619ccb53-f5bd-4435-815e-3e21249835e8
Delete of nonexisting embedding ID: 19088a4f-0616-4c58-8108-9d4b0bf2690d
Delete of nonexisting embedding ID: 0775eca1-720f-453f-b5d2-c5be3eebfd16
Delete of nonexisting embedding ID: b5747471-cdca-472a-b2c0-baed743d7255
Delete of nonexisting embedding ID: 7491b17a-d4f4-4879-97b5-bd15f199725e
Delete of nonexisting embedding ID: 7cbc935f-8b1f-41d7-9ead-7092207b6c7c
Delete of nonexisting embedding ID: d22196a9-840d-4700-9eae-85712fcc2851
Delete of nonexisting embedding ID: 41ce1ec5-9cc5-473e-ad7e-d9ae4e9709c9
Delete of nonexisting embedding ID: 08161580-e256-4f1a-b803-657e146f6ec4
Delete of nonexisting embedding ID: 4451a026-68fd-4242-88d9-fe19b52333c0
Delete of nonexisting embedding ID: 606f693b-ef30-4

✅ Base de datos creada con éxito.
📊 Número de documentos en la colección después de recargar: 33


### Ejercicio 9: Errores de Inserción 
 
Intenta crear un documento sin texto (ejemplo: cadena vacía). 
Corrige el error usando la función get_embeddings(). 
 
¿Qué lección aprendemos sobre los requisitos de ChromaDB?

In [31]:
# Intentar crear un documento con texto vacío
try:
    create_example("")
except Exception as e:
    print(f"Error al intentar crear un documento con texto vacío: {e}")

✅ Documento añadido con ID: 3c7bbe44-ffe0-4b45-9353-66881f179fe6


No dio error. No hay necesidad de corregir.